# Web Scraping Crypto Source & merge Kaggle dataset

In [ ]:
import pandas as pd

#Read in files
asset_details = pd.read_csv('/content/drive/MyDrive/Crypto Forecasting/asset_details.csv')
supplemental_train = pd.read_csv('/content/drive/MyDrive/Crypto Forecasting/supplemental_train.csv')
train = pd.read_csv('/content/drive/MyDrive/Crypto Forecasting/train.csv')

In [ ]:
#Create column for url scraper
asset_details['crypto_url'] = asset_details['Asset_Name']

#Convert column for url scraper
asset_details['crypto_url'].replace(to_replace = ['Bitcoin Cash', 'Binance Coin', 'EOS.IO', 
                                                  'Ethereum Classic', 'Ethereum', 'Litecoin',
                                                  'Monero', 'TRON', 'Stellar', 'Cardano', 'IOTA',
                                                  'Maker', 'Dogecoin', 'Bitcoin'],
                                    value = ['bitcoin-cash', 'binance-usd', 'eos', 
                                             'ethereum-classic','ethereum', 'litecoin',
                                             'monero', 'tron', 'stellar', 'cardano', 'iota',
                                             'maker', 'dogecoin', 'bitcoin'],
                                    inplace = True
                                   )
#Create list for scraper
Coin_Ticker = list(asset_details['crypto_url'])

In [ ]:
#Combine both datasets
concat_supp_train = pd.concat([supplemental_train, train],
                              ignore_index = True)

In [ ]:
#Merge asset details on asset tags
asset_supp_train = pd.merge(concat_supp_train, asset_details , on = ['Asset_ID'])

In [ ]:
#Create & convert new column
asset_supp_train['Time_stamp'] = asset_supp_train['timestamp'].astype('int')
asset_supp_train['Time_stamp'] = asset_supp_train['Time_stamp'].astype('datetime64[s]')
asset_supp_train.head(2)

In [ ]:
#Convert trading prices to USD
asset_supp_train['Open_USD']  = asset_supp_train['Open'].round(2)
asset_supp_train['High_USD']  = asset_supp_train['High'].round(2)
asset_supp_train['Low_USD']   = asset_supp_train['Low'].round(2)
asset_supp_train['Close_USD'] = asset_supp_train['Close'].round(2)

#Convert volume into millions
asset_supp_train['Volume_in_Millions'] = asset_supp_train['Volume'] / 1000000

#Check/drop any duplicate rowsp
asset_supp_train.drop_duplicates()

In [ ]:
#Function to scrape data on coins in wallet
def Asset_Name(wallet):
   
    crypto_data = pd.DataFrame()
    
    for coin in wallet:
        soup                = BeautifulSoup(HTML.text,'html.parser')
        coin_data           = soup.find_all("p",{"class":[]})

        #Crypto Assest Data
        Market_Cap_USD      = coin_data[0].text
        One_Day_Volume_USD  = coin_data[1].text
        Circulating_Supply  = coin_data[2].text
        Max_Supply          = coin_data[3].text
      

        data  = {
                 "crypto_url"          : Coin
                ,"Market_Cap_USD"      : Market_Cap_USD
                ,"One_Day_Volume_USD"  : One_Day_Volume_USD
                ,"Circulating_Supply"  : Circulating_Supply
                ,"Max_Supply"          : Max_Supply
            }

        crypto_data = crypto_data.append(data, ignore_index=True) 
                
    return crypto_data

In [ ]:
#Retrieve data on coins
Crypto_Data = Asset_Name(Coin_Ticker)
Crypto_Data

In [ ]:
#Merge Crypto Data with Kaggle dataframe
Crypto_coin_asset_supp_train = pd.merge(coin_asset_supp_train, Crypto_Data, how = 'left')